# 🗂️ 고성능 데이터셋 단위 테스트 (로깅 통합 버전)

이 노트북은 로깅 시스템이 통합된 고성능 데이터셋 테스트의 예시입니다.
모든 출력, 시각화, 결과가 체계적으로 `logs/unit_test/` 디렉토리에 저장됩니다.

## 📁 로그 저장 구조
```
logs/unit_test/highperf_dataset/20250905_143052/
├── logs/           # 텍스트 로그 및 출력
├── images/         # 시각화 결과
├── data/           # 처리된 데이터
├── results/        # 테스트 결과 JSON
└── test_summary.json
```

In [ ]:
import os
import sys

# 프로젝트 루트로 이동
print("현재 작업 디렉토리:", os.getcwd())
if 'notebooks' in os.getcwd():
    os.chdir("../")
print("변경 후 작업 디렉토리:", os.getcwd())

# 단위 테스트 로거 초기화
from src.utils.unit_test_logger import create_test_logger
test_logger = create_test_logger("highperf_dataset")
test_logger.log_info("고성능 데이터셋 단위 테스트 시작")

In [ ]:
# 필수 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
from torch.utils.data import DataLoader
import time
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 모듈 import
try:
    from src.data.dataset import HighPerfDocClsDataset
    from src.utils.common import load_yaml
    test_logger.log_success("모든 모듈 import 성공")
except Exception as e:
    test_logger.log_error("모듈 import 실패", e)
    raise

## 1. 📊 기본 데이터 분석

원본 데이터의 기본 정보를 분석하고 결과를 로깅합니다.

In [ ]:
# 출력 캡처를 사용하여 모든 print문을 로그 파일에 저장
with test_logger.capture_output("basic_data_analysis") as (output, error):
    print("=== 기본 데이터 분석 시작 ===")
    
    try:
        # 데이터 로드
        train_df = pd.read_csv("data/raw/train.csv")
        test_df = pd.read_csv("data/raw/meta.csv")
        
        print(f"✅ 학습 데이터: {len(train_df):,} 샘플")
        print(f"✅ 테스트 데이터: {len(test_df):,} 샘플")
        print(f"📊 클래스 수: {train_df['target'].nunique()}")
        
        # 클래스 분포 분석
        class_dist = train_df['target'].value_counts().sort_index()
        print(f"\n📊 클래스 분포:")
        for class_id, count in class_dist.head(10).items():
            print(f"   Class {class_id}: {count:,} 샘플 ({count/len(train_df)*100:.1f}%)")
        
        if len(class_dist) > 10:
            print(f"   ... 외 {len(class_dist)-10}개 클래스")
        
        # 기본 통계 저장
        basic_stats = {
            "train_samples": len(train_df),
            "test_samples": len(test_df),
            "num_classes": train_df['target'].nunique(),
            "class_distribution": dict(class_dist),
            "data_balance": {
                "min_samples": int(class_dist.min()),
                "max_samples": int(class_dist.max()),
                "std_samples": float(class_dist.std())
            }
        }
        
        test_logger.save_test_result("basic_data_analysis", {
            "status": "success",
            "stats": basic_stats
        })
        
        print("\n✅ 기본 데이터 분석 완료")
        
    except Exception as e:
        print(f"❌ 데이터 분석 실패: {e}")
        test_logger.save_test_result("basic_data_analysis", {
            "status": "failed",
            "error": str(e)
        })
        raise

# 데이터프레임을 파일로 저장
test_logger.save_dataframe(train_df.head(100), "sample_train_data", "학습 데이터 샘플 (100개)")
test_logger.save_dataframe(class_dist.to_frame('count'), "class_distribution", "클래스별 샘플 수")

In [ ]:
# 클래스 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# 클래스 분포 바 차트
class_dist.plot(kind='bar', ax=axes[0], color='skyblue', alpha=0.7)
axes[0].set_title('클래스별 샘플 수 분포', fontsize=14, fontweight='bold')
axes[0].set_xlabel('클래스 ID')
axes[0].set_ylabel('샘플 수')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# 클래스 불균형 정도 시각화
class_percentages = (class_dist / len(train_df) * 100).sort_values(ascending=False)
axes[1].pie(class_percentages.head(8), labels=[f'Class {i}' for i in class_percentages.head(8).index], 
           autopct='%1.1f%%', startangle=90)
axes[1].set_title('상위 8개 클래스 비율', fontsize=14, fontweight='bold')

plt.tight_layout()

# 그림을 로그 디렉토리에 저장
test_logger.save_figure(fig, "class_distribution_analysis", "클래스 분포 분석")
plt.show()
plt.close()

## 2. 🧪 데이터셋 클래스 테스트

HighPerfDocClsDataset 클래스의 기본 동작을 테스트합니다.

In [ ]:
with test_logger.capture_output("dataset_class_test") as (output, error):
    print("=== 데이터셋 클래스 테스트 시작 ===")
    
    try:
        # 설정 로드
        cfg = load_yaml("configs/train_highperf.yaml")
        print(f"✅ 설정 파일 로드 성공")
        
        # 테스트용 소규모 데이터셋 생성
        mini_train = train_df.groupby('target').head(3).reset_index(drop=True)
        mini_train_path = "temp_mini_train.csv"
        mini_train.to_csv(mini_train_path, index=False)
        
        print(f"📝 소규모 테스트 데이터셋 생성: {len(mini_train)} 샘플")
        
        # 테스트용 설정 수정
        test_cfg = cfg.copy()
        test_cfg['model']['img_size'] = 224
        test_cfg['training']['batch_size'] = 4
        
        # 데이터셋 생성 테스트
        dataset = HighPerfDocClsDataset(
            csv_file=mini_train_path,
            img_dir="data/raw/train",
            config=test_cfg,
            mode='train',
            fold=0,
            epoch=1
        )
        
        print(f"✅ 데이터셋 생성 성공: {len(dataset)} 샘플")
        
        # 샘플 데이터 로딩 테스트
        if len(dataset) > 0:
            sample_img, sample_label = dataset[0]
            print(f"✅ 샘플 데이터 로딩 성공")
            print(f"   이미지 크기: {sample_img.shape}")
            print(f"   이미지 타입: {type(sample_img)}")
            print(f"   레이블: {sample_label} (타입: {type(sample_label)})")
            
            # 이미지 텐서를 numpy로 변환하여 저장
            if isinstance(sample_img, torch.Tensor):
                img_np = sample_img.permute(1, 2, 0).numpy()
                # 정규화 해제 (0-1 범위로)
                img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())
                test_logger.save_numpy_array(img_np, "sample_image", "데이터셋에서 로드된 샘플 이미지")
        
        # DataLoader 테스트
        dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0)
        batch_img, batch_label = next(iter(dataloader))
        
        print(f"✅ DataLoader 테스트 성공")
        print(f"   배치 이미지 크기: {batch_img.shape}")
        print(f"   배치 레이블 크기: {batch_label.shape}")
        print(f"   배치 레이블: {batch_label.tolist()}")
        
        # 성능 메트릭 측정
        start_time = time.time()
        for i, (img, label) in enumerate(dataloader):
            if i >= 5:  # 5개 배치만 테스트
                break
        end_time = time.time()
        
        avg_time_per_batch = (end_time - start_time) / 5
        print(f"📊 평균 배치 로딩 시간: {avg_time_per_batch:.4f}초")
        
        # 테스트 결과 저장
        dataset_test_result = {
            "status": "success",
            "dataset_size": len(dataset),
            "sample_image_shape": list(sample_img.shape) if 'sample_img' in locals() else None,
            "batch_loading_time_sec": avg_time_per_batch,
            "config_used": {
                "img_size": test_cfg['model']['img_size'],
                "batch_size": test_cfg['training']['batch_size']
            }
        }
        
        test_logger.save_test_result("dataset_class_test", dataset_test_result)
        print("\n✅ 데이터셋 클래스 테스트 완료")
        
        # 임시 파일 정리
        os.remove(mini_train_path)
        
    except Exception as e:
        print(f"❌ 데이터셋 클래스 테스트 실패: {e}")
        test_logger.save_test_result("dataset_class_test", {
            "status": "failed",
            "error": str(e)
        })
        # 정리
        if os.path.exists(mini_train_path):
            os.remove(mini_train_path)
        raise

## 3. 🎨 Hard Augmentation 효과 분석

에포크별 Hard Augmentation 강도 변화를 분석하고 시각화합니다.

In [ ]:
with test_logger.capture_output("hard_augmentation_analysis") as (output, error):
    print("=== Hard Augmentation 분석 시작 ===")
    
    try:
        # 에포크별 증강 확률 계산
        total_epochs = 30
        epochs = list(range(1, total_epochs + 1))
        
        # 설정에서 증강 파라미터 가져오기
        aug_config = cfg.get('augmentation', {}).get('hard_augmentation', {})
        initial_prob = aug_config.get('initial_prob', 0.1)
        final_prob = aug_config.get('final_prob', 0.8)
        
        print(f"📊 Hard Augmentation 설정:")
        print(f"   초기 확률: {initial_prob}")
        print(f"   최종 확률: {final_prob}")
        print(f"   총 에포크: {total_epochs}")
        
        # 에포크별 확률 계산 (선형 증가)
        hard_aug_probs = []
        for epoch in epochs:
            progress = (epoch - 1) / (total_epochs - 1)
            prob = initial_prob + (final_prob - initial_prob) * progress
            hard_aug_probs.append(prob)
        
        print(f"\n📈 주요 에포크별 증강 확률:")
        for epoch in [1, 5, 10, 15, 20, 25, 30]:
            if epoch <= total_epochs:
                prob = hard_aug_probs[epoch-1]
                print(f"   Epoch {epoch:2d}: {prob:.3f} ({prob*100:.1f}%)")
        
        # 증강 확률 데이터 저장
        aug_prob_df = pd.DataFrame({
            'epoch': epochs,
            'hard_aug_probability': hard_aug_probs
        })
        
        test_logger.save_dataframe(aug_prob_df, "hard_augmentation_schedule", 
                                   "에포크별 Hard Augmentation 확률 스케줄")
        
        # 증강 강도 효과 분석
        intensity_analysis = {
            "initial_intensity": initial_prob,
            "final_intensity": final_prob,
            "intensity_increase": final_prob - initial_prob,
            "midpoint_epoch": total_epochs // 2,
            "midpoint_intensity": hard_aug_probs[total_epochs // 2 - 1]
        }
        
        test_logger.save_test_result("hard_augmentation_analysis", {
            "status": "success",
            "analysis": intensity_analysis,
            "schedule_type": "linear_progression"
        })
        
        print("\n✅ Hard Augmentation 분석 완료")
        
    except Exception as e:
        print(f"❌ Hard Augmentation 분석 실패: {e}")
        test_logger.save_test_result("hard_augmentation_analysis", {
            "status": "failed",
            "error": str(e)
        })
        raise

In [ ]:
# Hard Augmentation 스케줄 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. 에포크별 증강 확률 변화
axes[0, 0].plot(epochs, hard_aug_probs, 'b-', linewidth=2, marker='o', markersize=4)
axes[0, 0].set_title('에포크별 Hard Augmentation 확률 변화', fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Augmentation Probability')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='50% 기준선')
axes[0, 0].legend()

# 2. 증강 강도 구간별 분포
intensity_ranges = ['Low (0-0.3)', 'Medium (0.3-0.6)', 'High (0.6-1.0)']
range_counts = [
    sum(1 for p in hard_aug_probs if p < 0.3),
    sum(1 for p in hard_aug_probs if 0.3 <= p < 0.6),
    sum(1 for p in hard_aug_probs if p >= 0.6)
]

axes[0, 1].bar(intensity_ranges, range_counts, color=['lightgreen', 'orange', 'red'], alpha=0.7)
axes[0, 1].set_title('증강 강도별 에포크 분포', fontweight='bold')
axes[0, 1].set_ylabel('에포크 수')
for i, count in enumerate(range_counts):
    axes[0, 1].text(i, count + 0.5, str(count), ha='center', fontweight='bold')

# 3. 누적 증강 효과
cumulative_effect = np.cumsum(hard_aug_probs)
axes[1, 0].fill_between(epochs, cumulative_effect, alpha=0.5, color='purple')
axes[1, 0].plot(epochs, cumulative_effect, 'purple', linewidth=2)
axes[1, 0].set_title('누적 증강 효과', fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Cumulative Augmentation Effect')
axes[1, 0].grid(True, alpha=0.3)

# 4. 증강 강도 히트맵
intensity_matrix = np.array(hard_aug_probs).reshape(6, 5)  # 30 에포크를 6x5 격자로
im = axes[1, 1].imshow(intensity_matrix, cmap='viridis', aspect='auto')
axes[1, 1].set_title('증강 강도 히트맵 (6주 x 5일)', fontweight='bold')
axes[1, 1].set_xlabel('일 (Day)')
axes[1, 1].set_ylabel('주 (Week)')

# 컬러바 추가
cbar = plt.colorbar(im, ax=axes[1, 1], shrink=0.8)
cbar.set_label('Augmentation Probability')

plt.tight_layout()

# 시각화 결과 저장
test_logger.save_figure(fig, "hard_augmentation_schedule_visualization", 
                       "Hard Augmentation 스케줄 종합 분석")
plt.show()
plt.close()

## 4. 📊 성능 메트릭 및 최종 요약

In [ ]:
# 최종 성능 메트릭 수집
performance_metrics = {
    "data_loading": {
        "dataset_size": len(dataset) if 'dataset' in locals() else 0,
        "avg_batch_time_sec": avg_time_per_batch if 'avg_time_per_batch' in locals() else 0,
        "estimated_epoch_time_min": (avg_time_per_batch * len(train_df) / 4 / 60) if 'avg_time_per_batch' in locals() else 0
    },
    "augmentation": {
        "hard_aug_enabled": aug_config.get('enabled', False),
        "initial_prob": initial_prob if 'initial_prob' in locals() else 0,
        "final_prob": final_prob if 'final_prob' in locals() else 0,
        "avg_intensity": np.mean(hard_aug_probs) if 'hard_aug_probs' in locals() else 0
    },
    "data_quality": {
        "class_balance_std": float(class_dist.std()) if 'class_dist' in locals() else 0,
        "min_class_samples": int(class_dist.min()) if 'class_dist' in locals() else 0,
        "max_class_samples": int(class_dist.max()) if 'class_dist' in locals() else 0
    }
}

test_logger.save_performance_metrics(performance_metrics, "dataset_performance")

with test_logger.capture_output("final_summary") as (output, error):
    print("=== 최종 테스트 요약 ===")
    print(f"✅ 기본 데이터 분석: 완료")
    print(f"✅ 데이터셋 클래스 테스트: 완료")
    print(f"✅ Hard Augmentation 분석: 완료")
    print(f"✅ 성능 메트릭 수집: 완료")
    
    print(f"\n📊 주요 결과:")
    print(f"   총 학습 샘플: {len(train_df):,}개")
    print(f"   클래스 수: {train_df['target'].nunique()}개")
    print(f"   평균 배치 로딩 시간: {performance_metrics['data_loading']['avg_batch_time_sec']:.4f}초")
    print(f"   Hard Augmentation 평균 강도: {performance_metrics['augmentation']['avg_intensity']:.3f}")
    
    print(f"\n💡 권장사항:")
    if performance_metrics['data_quality']['class_balance_std'] > 500:
        print(f"   ⚠️ 클래스 불균형이 큽니다. 가중 샘플링 고려")
    if performance_metrics['data_loading']['avg_batch_time_sec'] > 0.1:
        print(f"   ⚠️ 데이터 로딩이 느립니다. num_workers 증가 고려")
    if performance_metrics['augmentation']['avg_intensity'] < 0.3:
        print(f"   💡 증강 강도가 낮습니다. 더 강한 증강 고려")
    
    print(f"\n🎯 다음 단계:")
    print(f"   1. test_mixup_augmentation.ipynb - Mixup 파라미터 최적화")
    print(f"   2. test_swin_model.ipynb - 모델 성능 벤치마크")
    print(f"   3. 실제 학습 파이프라인 실행")

# 테스트 완료
final_summary = test_logger.finalize_test()

print(f"\n🎉 모든 테스트 완료!")
print(f"📁 상세 결과: {test_logger.base_dir}")

## 🏆 테스트 결과 요약

### ✅ 완료된 테스트
1. **기본 데이터 분석**: 학습/테스트 데이터 크기, 클래스 분포 확인
2. **데이터셋 클래스 테스트**: HighPerfDocClsDataset 동작 검증
3. **Hard Augmentation 분석**: 에포크별 증강 강도 스케줄 분석
4. **성능 메트릭 수집**: 데이터 로딩 속도, 증강 효과 측정

### 📁 저장된 결과
- **로그 파일**: 모든 출력과 에러 메시지
- **시각화**: 클래스 분포, 증강 스케줄 차트
- **데이터**: 처리된 데이터프레임과 NumPy 배열
- **메트릭**: JSON 형태의 성능 지표

### 🔗 로그 디렉토리 접근
```bash
# 결과 확인
ls -la logs/unit_test/highperf_dataset/[timestamp]/

# 이미지 확인
ls logs/unit_test/highperf_dataset/[timestamp]/images/

# 테스트 요약 확인
cat logs/unit_test/highperf_dataset/[timestamp]/test_summary.json
```

이제 모든 테스트 결과가 체계적으로 저장되어 추후 분석과 비교가 가능합니다! 🎯